In [ ]:
import requests  # Python library to send HTTP requests.

url = "https://www.iisermohali.ac.in/"
#url= "https://timesofindia.indiatimes.com/"

response = requests.get(url)

# Check if request succeeded
if response.status_code == 200:
    html_content = response.text  # This is the raw HTML
    print("Successfully fetched the webpage!")
else:
    print("Failed to fetch the webpage. Status code:", response.status_code)


Successfully fetched the webpage!


In [4]:
from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')
# html
# ├── head
# │   └── title → 'IISER Mohali'
# └── body
#     ├── h1 → 'Welcome to IISER Mohali'
#     ├── p  → 'Indian Institute...'
#     └── p  → 'Established by...'
# We are assuming that all text content is within these tags.
# This is a very basic assumption and may not work for all websites.
# Best way to check is to inspect the website and see where the text lies.
# Extract text from common tags
valid_tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li'] # valid tags only from the html file, h->heading, p->paragraph, li->list
text = " ".join([element.get_text() for element in soup.find_all(valid_tags)]) # join the entire string into one.

print(text[:])  # print characters


 IISER MOHALI Institute Academics Research People Outreach 79th Independence Day Celebration at IISER Mohali International Yoga Day Celebration at IISER Mohali IISER Mohali awarded degrees to 316 Graduating Students IISER Mohali Science Festival - Inaugurated by Hon'ble Education Minister of Punjab Science demonstrations & competitions The Hon'ble Vice-President of India interacts with IISER Mohali Faculty, Students & Staff Visit of Hon'ble Vice-President of India to IISER Mohali Tattva 2025 Science Fest - Celebrating the essence of Scientific Curiosity  76th Republic Day Celebration at IISER Mohali 500+ School students at IISER Mohali for National Space Day Annual Science Festival Lectures by Dr. Anjan Ray & Dr. Srivari Chandrashekhar Students awarded for outstanding performance in Academics & Sports FACULTY ACHIEVEMENTS PROFESSOR ANIL KUMAR TRIPATHI Director, IISER Mohali Elected as Vice President (Science Promotion) of INSA  1800 school students across tricity visited IISER Mohali  

Now after getting the 'text' part , our next step is to clean the text we get after parsing.
This includes getting rid of symbols , punctuations and numbers and converting all the text to lower case, so that we can standardize the text for word frequency analysis . 


In [ ]:
import re

def clean(text:str) -> str:

   text = text.lower() #convert text to lower case

   text =re.sub(r'[^a-z\s]', ' ',text)

   text = re.sub(r'\b(?![ai]\b)[a-z]\b',' ',text)

   text = re.sub(r'\s+',' ',text).strip()

   return text

clean(text)

cleaned_text = clean(text)


##### Now we will break the string into single words and count the frequencies.
##### That is, tokenize, count and store.

In [ ]:
from collections import Counter

def count_words(cleaned_text: str) -> Counter:
    """
    Takes a cleaned text string (only lowercase letters and spaces),
    returns a Counter mapping word -> frequency.
    """

    words = cleaned_text.split() # splits the cleaned string over white spaces

    word_counter = Counter(words) # gets frequency of words   

    return word_counter

Counted_words = count_words(cleaned_text)
most_used = Counted_words.most_common(20)

print("Top 10 most used words:")
for word,freq in most_used:
    print(f"{word:>12} : {freq}")


Top 10 most used words:
       iiser : 15
      mohali : 14
          of : 8
     science : 6
    sciences : 6
   committee : 6
    research : 5
         day : 5
    students : 5
   institute : 4
          at : 4
     faculty : 4
        news : 4
   academics : 3
 celebration : 3
          by : 3
         hon : 3
         ble : 3
         the : 3
        vice : 3


Now, we need to store data. It is database where we can query, filter and join across multiple sites later.

CSV is static: you can only store and read rows.
SQLite is dynamic: you can query — “show words used more than 50 times,” or “compare IISER Mohali vs Pune.”

In [ ]:
import sqlite3

def store_sqlite(word_counts: Counter, db_path='words.db', table='word_frequency'):
    # word_counts: Counter — our counted data from earlier (Counter is like a dict: {word: freq}).
    # db_path='words.db' — the name of the database file.
    # table='word_frequency' — the name of the SQL table to store data in.

    # Connect to the database 
    conn = sqlite3.connect(db_path) # Opens (or creates) a database file. conn is the returned object here -> active connection to that database.
    cur = conn.cursor() # Gives you a cursor, which is like a “pen” used to execute SQL commands.
    #SQL databases work transactionally — you make changes through a cursor, then commit them all at once.

    # Create table
    cur.execute(f"""
        CREATE TABLE IF NOT EXISTS {table} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            word TEXT UNIQUE,
            frequency INTEGER
        )
    """)
        # id INTEGER PRIMARY KEY AUTOINCREMENT → auto numbering for each row.
        # word TEXT UNIQUE → ensures no duplicates (each word appears once).
        # frequency INTEGER → stores how often the word appears.
        # SQL needs data types. TEXT for words, INTEGER for counts.

    # updating the table, putting values in it.
    for i , freq in word_counts.items(): # loop over all wordandfreqency pairs
        cur.execute(f"""
            INSERT INTO {table} (word , frequency)
            VALUES (? , ?)
            ON CONFLICT(word) DO UPDATE SET frequency = excluded.frequency
        """, (i,freq))
        # executing againusing the cursor, 
        # INSERT INTO {table} (word , frequency) VALUES (? , ?) ---->>>> adds a new row
        # ? -> replaced by (i, freq), 
        # ON CONFLICT(word) DO UPDATE --->>> if same word then update earlier thing.

    ## save the changes.
    conn.commit() # permanently writes all pending changes to disk.
    conn.close() # ends the database session freeign the used resources.


In [ ]:
## now to read the stored data back, we proceed as.

def read_sql(db_path='words.db', table='word_frequency', top=20):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(f"SELECT word , frequency FROM {table} ORDER BY frequency DESC LIMIT ?", (top,))
    rows = cur.fetchall()
    conn.close
    return rows  

## It do as:
# SELECT word, frequency
# FROM word_frequency
# ORDER BY frequency DESC
# LIMIT top = 20;  

In [ ]:
store_sqlite(Counted_words)
TOP_20 = read_sql()
for w, f in TOP_20:
    print(f"{w:>12}  {f}")

       iiser  15
      mohali  14
          of  8
     science  6
    sciences  6
   committee  6
    research  5
         day  5
    students  5
   institute  4
          at  4
     faculty  4
        news  4
   academics  3
 celebration  3
          by  3
         hon  3
         ble  3
         the  3
        vice  3


#### Now we sqlite3 DB, we need to store multiple sites data into it, iteratively or seperately.